In [1]:
import fitz
import pandas as pd 

In [2]:
doc = fitz.open('protocols/31325195.pdf')
for page in doc:
    words = page.get_text("words")
actual_words = []
for i in words:
    actual_words.append(i[4])

In [186]:
def hemmalag_words(words):
    start = None
    stop = None
    count_bortalag = 0
    for index, word in enumerate(words):
        if  word == "OT:":
            start = index+2 
        if word == "Bortalag:":
            count_bortalag += 1
            stop = index
            if count_bortalag == 2:
                break
    hem = words[start:stop]
    return hem

def bortalag_words(words):
    start = None
    stop = None
    for index, word in enumerate(words):
        if  word == "OT:":
            start = index+2 
        if word == "Actions":
            stop = index
            break
    bort = words[start:stop]
    return bort

def get_first_row(lag):
    lag = lag[5:]
    points = None
    for index,value in enumerate(lag):
        if value.isnumeric() and lag[index+2].isnumeric():
            points = index
            first_row = {
                "player_name": " ".join(lag[1:index]),
                "points": lag[index],
                "fouls": lag[index+1]
            }
            break
    return first_row

def clear_players_table(t_team):
    indexes = []
    player_rows = [get_first_row(t_team)]
    for index, value in enumerate(t_team):
        if value.isnumeric() and index+2<len(t_team) and t_team[index+2].isnumeric() and t_team[index+2] != 'C':
            indexes.append(index+3)
    for i,d in enumerate(indexes):
        player_rows.append(
            {
                "player_name": " ".join(t_team[d:indexes[i+1]-3]) if i+1<len(indexes) else " ".join(t_team[d:d+2]),
                "points": t_team[indexes[i+1]-3] if i+1<len(indexes) else t_team[d+2],
                "fouls": t_team[indexes[i+1]-2] if i+1<len(indexes) else t_team[d+3]
            }
        )
    return player_rows

def csv_lag(lag):    
    rows = []
    ny_lag = [x for x in lag if x not in ['X', 'Ø']]
    player_stats = clear_players_table(ny_lag)
    lag_df = pd.DataFrame(player_stats)
    return lag_df

def get_period_data(words: list, period: int):
    start = None
    stop = None
    for index, word in enumerate(words):
        if  word == "Start" and words[index+2]== str(period):
            start = index+3
        if period != 4:
            if word == "Start" and words[index+2]== str(period+1):
                stop = index
                break
        else:
            if word == "Scores":
                stop = index
                break            
    period_data = words[start:stop]
    return period_data

def clean_period_data(actual_words: list):
    period_data = []
    timeouts = []
    for period in range(1,5):
        period_data += get_period_data(words=actual_words, period=period)
    timeouts = [index-2 for index,element in enumerate(period_data) if element == 'timeout']
    count=0
    for i in range(len(timeouts)):
        period_data.insert(timeouts[i]+count, "timeout!")
        count += 1
    period_rows = []
    for i in range(0,len(period_data),7):
        period_rows.append(
            {
                "score": period_data[i]+period_data[i+1]+period_data[i+2],
                "player": period_data[i+3],
                "team":  period_data[i+4] + '' + period_data[i+5],
                "basket_count": period_data[i+6]
            }
        )
    return pd.DataFrame(period_rows)


In [187]:
hem = hemmalag_words(actual_words)
hemmalag_csv = csv_lag(hem)
bort = bortalag_words(actual_words)
bortalag_csv = csv_lag(bort)
match = clean_period_data(actual_words)
hemmalag_csv

,player_name,points,fouls
0,Robert Jonsson,11,1
1,Antonio Rahme,0,0
2,Sebastian Jonsson,12,2
3,Georgiy Gartman,15,4
4,Senne Willems,0,2
5,Don-divin Ngenda Rwemera,0,1
6,Anton Stenman,37,1
7,Tarik Curkic,0,0
8,Dominiq Salo,2,2
9,Charbel Rahme,12,4


In [189]:
bortalag_csv



,player_name,points,fouls
0,Evan Thomas,26,5
1,Jacob Birgersson,9,0
2,Lars Olsson,6,5
3,Christos Marinos,2,1
4,Ajay Bhoolokam,6,3
5,Antonios Pekopoulos,18,2
6,Johannes Hüffmeier,6,3
7,David Solberger,7,2
8,Johan Ericsson,2,2
9,Christoffer Kopp,6,3


In [190]:
match

,score,player,team,basket_count
0,2-0,#12,LagA,2p
1,4-0,#10,LagA,2p
2,5-0,#10,LagA,1p
3,7-0,#4,LagA,2p
4,7-3,#6,LagB,3p
...,...,...,...,...
106,95-86,#14,LagA,1p
107,97-86,#6,LagA,2p
108,97-88,#5,LagB,2p
109,98-88,#10,LagA,1p
